In [38]:
#! pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 51.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 51.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
#import one hot encoder
from sklearn.preprocessing import OneHotEncoder
# suppress warnings
import warnings
warnings.filterwarnings("ignore")
import xgboost

import sys
import os
import pickle

sys.path.append(os.path.join(os.path.dirname('Library/data_prep.py'), 'lib'))

In [40]:
import sys
sys.path.append('/workspaces/Machine_Learning_G19/Library')

from data_prep import PreProcessor1

In [41]:
pr = PreProcessor1()

In [42]:
# import a pickle file
filename = 'model.pkl'

os.chdir('../Models')

pr = pickle.load(open('prepProcessor41.pkl', 'rb'))
model = pickle.load(open('7final_model.pkl', 'rb'))

os.chdir('../Data')

test = pd.read_csv('test_data.csv')

In [43]:
test = pd.read_csv('test_data.csv')

In [45]:
pr.status
pr.update_status('test')

print("------------updates---------------------")
pr.update_casted_cols()
print("------------set_casting---------------------")
pr.set_castings(test)
print("------------pipeline---------------------")
df_test = pr.cast_pipeline(test)
print("df after cast_pipeline:", df_test.shape)
print("------------fillna---------------------")
pr.update_fillna_list(df_test)   
df_test = pr.fillna_pipeline(df_test)
print("df after fillna_pipeline:", df_test.shape)
print("------------transformation---------------------")
pr.set_transformations(df_test)
df_test = pr.transformation_pipeline(df_test)
print("df after transformation_pipeline:", df_test.shape)
df_test = pr.scaling_pipeline(df_test)
df_test.info()


------------updates---------------------
------------set_casting---------------------
Columns that are not casted:
-fe_County of Injury: float64
-fe_Carrier Type: float64
-Assembly Year: Int64
-Claim Identifier: int64
-C-3 Month: Int64
-First Hearing Month: Int64
-C-3 Year: Int64
-Assembly Month: Int64
-Accident Month: Int64
-log_Average Weekly Wage: float64
-fe_Medical Fee Region: float64
-First Hearing Year: Int64
-Age: Int64
-Accident Year: Int64
------------pipeline---------------------
Feature Carrier Type is already in scaling list
'Agreement Reached'
-Column Agreement Reached not found
Feature Zip Code is already in scaling list
'Accident Date'
-Column Accident Date not found
'Assembly Date'
-Column Assembly Date not found
'First Hearing Date'
-Column First Hearing Date not found
"['Industry Code Description'] not found in axis"
-Column Industry Code Description not found
"['OIICS Nature of Injury Description'] not found in axis"
-Column OIICS Nature of Injury Description not fo

In [46]:
model.feature_importances_

array([0.15893859, 0.59979177, 0.01673862, 0.19234851, 0.        ,
       0.02909366, 0.00308889], dtype=float32)

In [50]:
cols = ['Age', 'Average Weekly Wage', 'COVID-19 Indicator', 'fe_Medical Fee Region', 'Gender_X', 'Gender_U', 'Alternative Dispute Resolution']
x = model.predict(df_test[cols])

In [51]:
len(x)

387975

In [52]:
with open('model_experiment.pkl', 'wb') as f:
    # Serialize both the list and the object
    pickle.dump(cols, f)
    pickle.dump(pr, f)

In [53]:
with open('model_experiment.pkl', 'rb') as f:
    loaded_list = pickle.load(f)
    loaded_object = pickle.load(f)
    
print(loaded_list)

print(loaded_object)



['Age', 'Average Weekly Wage', 'COVID-19 Indicator', 'fe_Medical Fee Region', 'Gender_X', 'Gender_U', 'Alternative Dispute Resolution']
PreProcessor1: testscaler: RobustScaler()version: 4.1 lackOfFrango
